In [19]:
import numpy as np
from easymocap.mytools.camera_utils import write_camera, read_camera
import json
import cv2
import os
import matplotlib.pyplot as plt
import shutil
from tqdm.auto import tqdm

folder = 'data/wildtrack/'
output_folder = 'data/wildtrack_shifted/'


pids = [0,1,2,3,7,9,10]
# join the masks together and invert them
cameras = read_camera("data/wildtrack/intri.yml", "data/wildtrack/extri.yml", [1,2,3,4,5,6,7])

new_world_origin = np.array([4.5,12.5,0])

def get_json(file):
    with open(file) as f:
        data = json.load(f)
    return data

def save_json(file, data):
    with open(file, 'w') as f:
        json.dump(data, f, indent=4)
        
def shift_kp3d(kp3d_folder, shift, output_kp3d_folder):
    for file in os.listdir(kp3d_folder):
        if file.endswith(".json"):
            data = get_json(kp3d_folder + file)
            data_new = []
            for person in data:
                assert person["id"] in pids, f"Person ID {person['id']} not in pids"
                keypoints3d = []
                for keypoint in person["keypoints3d"]:
                    keypoints3d.append([keypoint[0] - shift[0], keypoint[1] - shift[1], keypoint[2] - shift[2]])
                data_new.append({"id": person["id"], "keypoints3d": keypoints3d})
            save_json(output_kp3d_folder + file, data_new)
            
def shift_smpl(smpl_folder, shift, output_smpl_folder):
    for file in os.listdir(smpl_folder):
        if file.endswith(".json"):
            data = get_json(smpl_folder + file)
            data_new = []
            for person in data:
                assert person["id"] in pids, f"Person ID {person['id']} not in pids"
                th = person["Th"]
                th[0][0] -= shift[0]
                th[0][1] -= shift[1]
                th[0][2] -= shift[2]
                data_new.append({"id": person["id"], "Rh":person["Rh"], "Th": th, 
                                 "poses": person["poses"], "shapes": person["shapes"]})
            save_json(output_smpl_folder + file, data_new)         

def shift_vertices(vertices_folder, shift, output_vertices_folder):
    for file in os.listdir(vertices_folder):
        if file.endswith(".json"):
            data = get_json(vertices_folder + file)
            data_new = []
            for person in data:
                assert person["id"] in pids, f"Person ID {person['id']} not in pids"
                vertices = []
                for vertex in person["vertices"]:
                    vertices.append([vertex[0] - shift[0], vertex[1] - shift[1], vertex[2] - shift[2]])
                data_new.append({"id": person["id"], "vertices": vertices})
            save_json(output_vertices_folder + file, data_new)

for dir in tqdm(os.listdir(folder)):
    print(dir)
    if dir == "output-keypoints3d":
        if "keypoints3d" in os.listdir(folder + dir):
            os.makedirs(output_folder + dir + "/" + "keypoints3d", exist_ok=True)
            shift_kp3d(folder + dir + "/" + "keypoints3d/", new_world_origin, output_folder + dir + "/" + "keypoints3d/")
        if "label" in os.listdir(folder + dir):
            shutil.copytree(folder + dir + "/" + "label", output_folder + dir + "/" + "label", dirs_exist_ok=True)
    if dir == "output-smpl-3d":
        if "instance" in os.listdir(folder + dir):
            os.makedirs(output_folder + dir + "/" + "instance", exist_ok=True)
            shift_kp3d(folder + dir + "/" + "instance/", new_world_origin, output_folder + dir + "/" + "instance/")
        if "smpl" in os.listdir(folder + dir):
            os.makedirs(output_folder + dir + "/" + "smpl", exist_ok=True)
            shift_smpl(folder + dir + "/" + "smpl/", new_world_origin, output_folder + dir + "/" + "smpl/")
        if "vertices" in os.listdir(folder + dir):
            os.makedirs(output_folder + dir + "/" + "vertices", exist_ok=True)
            shift_vertices(folder + dir + "/" + "vertices/", new_world_origin, output_folder + dir + "/" + "vertices/")
    if dir == "mask-ignore":
        shutil.copytree(folder + dir, output_folder + dir, dirs_exist_ok=True)
    if dir == "mask-background":
        shutil.copytree(folder + dir, output_folder + dir, dirs_exist_ok=True)
    if dir == "images":
        shutil.copytree(folder + dir, output_folder + dir, dirs_exist_ok=True)
    
for cam in cameras["basenames"]:

    T = cameras[cam]["T"]
    R = cameras[cam]["R"]
    C_old = -np.dot(R.T, T)
    C_new = C_old - np.array([new_world_origin]).reshape(3,1)
    T_new = -np.dot(R,C_new)
    cameras[cam]["T"] = T_new
    del cameras[cam]["H"]
    del cameras[cam]["W"]
del cameras["basenames"]   
write_camera(cameras,output_folder)
    

  0%|          | 0/7 [00:00<?, ?it/s]

extri.yml
images
intri.yml
mask-background
mask-ignore
output-keypoints3d
output-smpl-3d


In [9]:
cameras

{'1': {'K': array([[1.74344788e+03, 0.00000000e+00, 9.34520203e+02],
         [0.00000000e+00, 1.73515662e+03, 4.44398773e+02],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  'invK': array([[ 5.73576081e-04,  0.00000000e+00, -5.36018436e-01],
         [ 0.00000000e+00,  5.76316853e-04, -2.56114502e-01],
         [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]]),
  'H': 0,
  'W': 0,
  'RT': array([[ 0.87767617,  0.47858941,  0.02523331, -5.258942  ],
         [ 0.1338929 , -0.19430829, -0.97175973,  0.454076  ],
         [-0.46017087,  0.85626892, -0.23461948,  9.867235  ]]),
  'R': array([[ 0.87767617,  0.47858941,  0.02523331],
         [ 0.1338929 , -0.19430829, -0.97175973],
         [-0.46017087,  0.85626892, -0.23461948]]),
  'Rvec': array([[ 1.759099],
         [ 0.467101],
         [-0.331699]]),
  'T': array([[-5.258942],
         [ 0.454076],
         [ 9.867235]]),
  'center': array([[12.31185224]]),
  'P': array([[ 1.10014368e+03,  1.63459629e+03, -1.7526